# Higher-order functions

Higher-order functions (HOFs) are functions which receive or return other functions. HOFs show that functions can be regarded as _values_, much in the same way than integers, booleans, product values, sum values, etc. We will see that higher-order functions are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

## Functions as values

As we saw, functions are represented through methods in Scala (as in any other object-oriented language). But methods themselves can't be passed around and be returned by invocations. Therefore, the first thing to do in order to create HOFs in Scala is finding a way to reify methods. For instance, let's consider the following functions:

In [ ]:
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

We want to implement a function that receives an integer-to-integer function, such as `addOneM`and `substractOneM`, and call this function over a given number. We may want to write something like this:

In [ ]:
// def call(int2int(number: Int): Int, number: Int): Int = ???

where the first argument `int2int` attempt to represent any function that receives an integer and returns another integer. 

This code is not legal in Scala, but we can create a new class whose only method is the function that we want to pass around:

In [ ]:
abstract class FunctionInt2Int{
    def apply(number: Int): Int
}

Now, we can implement the `call` HOF as follows: 

In [ ]:
def call(int2int: FunctionInt2Int, number: Int): Int = 
    int2int.apply(number)

In order to use this HOF with the `addOneM` and `substractOneM` functions, we must create reified versions for them: 

In [ ]:
val addOneV: FunctionInt2Int = new FunctionInt2Int{
    def apply(number: Int): Int = 
        number + 1
}

val substractOneV: FunctionInt2Int = new FunctionInt2Int{
    def apply(number: Int): Int = 
        number - 1 
}

We call the `addOneV` and `substractOneV` function-values, i.e. functions represented as values. Now, we can use the `call` HOF as follows:

In [ ]:
assert(call(addOneV, 5) == 6)

In [ ]:
call(substractOneV, 5)

### Standard functions in Scala

The Scala programming language offers many facilities to work with functions as values. First, the standard library provides the following _generic_ types [`Function1`](https://www.scala-lang.org/api/current/scala/Function1.html), [`Function2`](https://www.scala-lang.org/api/current/scala/Function2.html), ...:

In [ ]:
trait Function1[A, B]{
    def apply(a: A): B
}

trait Function2[A, B, C]{
    def apply(a: A, b: B): C
}

// up to Function22

Using these standard classes, we can create the `addOneV` function-value in a similar way: 

In [ ]:
val addOneV: Function1[Int, Int] = new Function1[Int, Int]{
    def apply(a: Int): Int = 
        a + 1
}

But we can do it more easily, since Scala also provides special syntax to declare function types and create functions (so-called _lambda expressions_):

In [ ]:
val addOneV: Int => Int = 
    (a: Int) => a + 1

val substractOneV: Int => Int = 
    (a: Int) => a - 1

And we can also profit from type inference:

In [ ]:
val addOneV: Int => Int = 
    a => a + 1

val substractOneV: Int => Int = 
    a => a - 1

Using these syntactic facilities we can write the `call` HOF more easily: 

In [ ]:
def call(int2int: Int => Int, number: Int): Int = 
    // int2int.apply(number)
    int2int(number)

which we can use as follows:

In [ ]:
call(addOneV, 1)
call(substractOneV, 1)

We can even pass function-methods that are converted on the fly to function-values (this is the so-called _eta-expansion_):

In [ ]:
call(addOneM, 1)
call(substractOneM, 1)

Finally, for functions of two arguments we can use a similar syntax as well. So, instead of writing the more verbose:

In [ ]:
val sum: Function2[Int, Int, Int] = new Function2[Int, Int, Int]{
    def apply(a: Int, b: Int): Int = 
         a + b
}

we can create a lambda expressions for a `Function2` value in the following way:

In [ ]:
val sum: (Int, Int) => Int = 
    (a: Int, b: Int) => a + b

or, exploiting type inference:

In [ ]:
val sum: (Int, Int) => Int = 
    (a, b) => a + b 

### Improved unit testing

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

Using higher-order functions we can create test catalogues which are parameterised by the function to be tested. For instance, let's consider the following two alternative implementations:

In [ ]:
// Recursively

def sumR(list: List[Int]): Int = 
    list match {
        case Nil => 0 : Int
        case head :: tail => head + sumR(tail) : Int 
    }

In [ ]:
// With tail-recursion

def sumTR(list: List[Int]): Int = {

    def sumAux(acc: Int, list: List[Int]): Int = 
        list match {
            case Nil => acc : Int
            case head :: tail => sumAux(head + acc, tail) : Int 
        }
    
    sumAux(0, list)
}

Instead of creating ad-hoc test catalogues for each alternative, we can create a single one that receives the function to be tested as argument as follows:

In [ ]:
/*
object TestSumR extends FlatSpec with Matchers{
    "length" should "work" in {
        sumR(List()) shouldBe 0 
        sumR(List(1)) shouldBe 1 
        sumR(List(1,2,3,4)) shouldBe 10
    }
}
*/

class TestSum(sum: List[Int] => Int) extends FlatSpec with Matchers{
    "length" should "work" in {
        sum(List()) shouldBe 0 
        sum(List(1)) shouldBe 1 
        sum(List(1,2,3,4)) shouldBe 10
    }
}

Now, we can test the `sumR` and `sumTR` functions by reusing the same test catalogue:

In [ ]:
run(new TestSum(sumR))

In [ ]:
run(new TestSum(sumTR))

## Functions compose

We can create new functions by composing other functions whose signatures match. This is great from a modularity perspective. For instance, the following function is implemented in a non-modular way:

In [ ]:
def isEvenLength: String => Boolean = 
    (s: String) => s.length % 2 == 0

This function is somehow the combination of two more basic functions `length` and `isEven`:

In [ ]:
def length: String => Int = 
    s => s.length

In [ ]:
def isEven: Int => Boolean = 
    i => i % 2 == 0

but this is not reflected in the current implementation. How can we redefine the function `isEvenLength` from the functions `length` and `isEven`? We can use a HOF which is able to compose functions:

In [ ]:
def compose[A, B, C](f2: B => C, f1: A => B): A => C = 
    (a: A) => f2(f1(a))

Then, we can redefine `isEvenLength` in a modular way from the `length` and `isEven` building blocks:

In [ ]:
val isEvenLength: String => Boolean = 
    compose(isEven, length)

The HOF `compose` is actually defined by `Function1`: 

In [ ]:
val isEvenLength: String => Boolean = 
    isEven.compose(length)

or using infix notation:

In [ ]:
val isEvenLength: String => Boolean = 
    isEven compose length

Note that a similar function to `compose`, called `andThen`, is also available: 

In [ ]:
val isEvenLength: String => Boolean = 
    length andThen isEven

Also, there is a function which behaves as the identity element with respect to the operation `compose`:

In [19]:
def identity[A]: A => A = 
    (a: A) => a

defined function identity

No matter which other function we choose to compose with the `identity` function, the result will be that function:
1. `identity[B] compose f == f` for all `f: A => B`
2. `f compose identity[A] == f` for all `f: A => B`

## HOFs as modularity devices

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [20]:
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }

defined function sum

In [21]:
def multiply(list: List[Int]): Int = 
    list match {
        case Nil => 1
        case head :: tail => head * multiply(tail)
    }

defined function multiply

These functions clearly share a common logic; their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [22]:
def combine(list: List[Int])(nil: Int, cons: (Int, Int) => Int): Int = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, combine(tail)(nil, cons))
    }

defined function combine

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [23]:
def sum(list: List[Int]): Int = 
    combine(list)(0, (a, b) => a + b)

defined function sum

In [24]:
def multiply(list: List[Int]): Int = 
    combine(list)(1, _ * _)

defined function multiply

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`'s):

In [25]:
def foldRight[A, B](list: List[A])(nil: B, cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil, cons))
    }

defined function foldRight

The implementation of `sum` and `multiply` using `foldRight` is no more difficult:

In [26]:
def sum(list: List[Int]): Int = 
    foldRight[Int, Int](list)(0, _ + _)

defined function sum

In [ ]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a: Int, b: Int) => a * b)

although you may have noticed that we have to give extra type information in the invocations to `foldRight`. In fact, the following code doesn't compile. Check it yourself!

In [ ]:
/*
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
*/

In order to help the Scala compiler to infer the type parameters of the `foldRight` function, we need to change its signature a little bit: 

In [27]:
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }

defined function foldRight

By splitting the second parameter list, we allow the Scala compiler to infer the type of `B`, before it analyses the type of the `cons` argument. Now this works:

In [ ]:
foldRight(List(1,2,3))(1)(_*_)

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [1]:
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

defined function foldLeft
defined function filter
defined function map
defined function flatMap

But most of these functions do not work only for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance, for `Option`:

In [5]:
def fold[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

In [6]:
def fold[A, B, C](opt: Either[A, B])(left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (we will see catamorphisms later on). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. In the following implementation, its signature declares the initial value of a mutable variable, and the update function executed in every step of the loop:

In [15]:
def foldLeft[A, B](list: List[A])(initial: B)(update: (B, A) => B): B = {
    var out: B = initial
    for (elem <- list) 
        out = update(out, elem)
    out
}

defined function foldLeft

Also, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [31]:
@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    list match {
        case Nil => out
        case head :: tail => 
            foldLeft(tail)(update(out, head))(update)
    }

defined function foldLeft

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are one-liner implementations of many of the functions of the last notebook on recursive functions:

In [32]:
def length[A](list: List[A]): Int = 
    foldLeft(list)(0)((out: Int, _: A) => out + 1)

defined function length

In [33]:
def sum(list: List[Int]): Int = 
    foldLeft(list)(0)(_ + _)

defined function sum

In [34]:
def reverse[A](list: List[A]): List[A] = 
    foldLeft(list)(List[A]())((out: List[A], a: A) => a :: out)

defined function reverse

## The `map` function

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements are not modified. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

The implementation for lists can be done as follows:

In [28]:
def map[A, B](list: List[A])(f: A => B): List[B] = 
    foldRight(list)(Nil: List[B])((a, l) => f(a) :: l)

defined function map

In [29]:
map(List(1,2,3))(_.toString)

res28: List[String] = List("1", "2", "3")